In [18]:
# ✅ 修复方案2：诊断qlib数据源问题
print("=== 诊断qlib数据源 ===")

# 检查qlib是否能获取instruments
try:
    result = D.instruments()
    print(f"D.instruments()返回类型: {type(result)}")
    print(f"是否为None: {result is None}")
    
    if result is None:
        print("❌ D.instruments()返回None，这说明数据源有问题")
        print("可能的原因：")
        print("1. qlib数据路径不存在或无效")
        print("2. instruments.txt文件缺失或格式错误")
        print("3. qlib未正确初始化")
        
        # 尝试手动指定ETF列表
        print("\n使用手动指定的ETF列表:")
        sample_df = D.features(
            instruments=etf_list[:1],  # 只测试第一只
            fields=["$close", "$volume"],  # 指定具体字段
            start_time="2023-01-01", 
            end_time="2023-01-02"
        )
        print(f"✅ 手动指定成功: {sample_df.shape}")
        print(sample_df)
        
        
    else:
        print(f"✅ D.instruments()成功返回: {len(result) if hasattr(result, '__len__') else 'N/A'}")
        if hasattr(result, '__len__') and len(result) > 0:
            print(f"前5个instruments: {list(result)[:5]}")
            
except Exception as e:
    print(f"❌ D.instruments()调用失败: {e}")
    print("这说明qlib可能没有正确初始化")


=== 诊断qlib数据源 ===
D.instruments()返回类型: <class 'dict'>
是否为None: False
✅ D.instruments()成功返回: 2
前5个instruments: ['market', 'filter_pipe']


In [19]:
# ✅ 修复方案3：使用项目中的数据转换工具
print("=== 使用项目数据转换工具 ===")

# 如果你有ETF数据文件，可以使用项目中的转换工具
import sys
import os
sys.path.append('../qlib_workflow')

try:
    from data_converter import ETFDataConverter
    
    print("正在检查ETF数据转换器...")
    converter = ETFDataConverter()
    
    # 如果你有原始ETF数据，可以运行转换
    # result = converter.run()
    # print(f"数据转换完成: {result}")
    
except ImportError as e:
    print(f"无法导入数据转换器: {e}")
except Exception as e:
    print(f"数据转换器执行失败: {e}")

# 最终的正确用法示例
print("\n=== 正确的D.features()用法 ===")
print("重要提示：")
print("1. instruments 参数必须是列表，即使只有一个股票也要用 [stock_code]")
print("2. qlib必须正确初始化并有有效的数据源")
print("3. 如果D.instruments()返回None，说明数据源有问题")

print("\n正确的代码格式：")
print("sample_df = D.features(")
print("    instruments=[etf_list[0]],  # 列表格式！")
print("    fields=None,")  
print("    start_time='2023-01-01',")
print("    end_time='2023-01-02'")
print(")")


=== 使用项目数据转换工具 ===
正在检查ETF数据转换器...

=== 正确的D.features()用法 ===
重要提示：
1. instruments 参数必须是列表，即使只有一个股票也要用 [stock_code]
2. qlib必须正确初始化并有有效的数据源
3. 如果D.instruments()返回None，说明数据源有问题

正确的代码格式：
sample_df = D.features(
    instruments=[etf_list[0]],  # 列表格式！
    fields=None,
    start_time='2023-01-01',
    end_time='2023-01-02'
)


In [26]:
import qlib
from qlib.data import D
import pandas as pd

# 初始化qlib，假设数据已在~/data/qlib_data/etf_data
qlib.init(provider_uri="/Users/liyuan/data/qlib_data/etf_data/",region="cn")

[1408:MainThread](2025-08-07 00:08:44,701) INFO - qlib.Initialization - [config.py:451] - default_conf: client.
[1408:MainThread](2025-08-07 00:08:44,705) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[1408:MainThread](2025-08-07 00:08:44,706) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': PosixPath('/Users/liyuan/data/qlib_data/etf_data')}


In [28]:
# 假设etf列表如下
etf_list = [
    "560590.SH",
    "159680.SZ",
    "561780.SH",
    "159677.SZ",
    "159685.SZ"
]


# 取一只ETF，获取其所有字段
sample_df = D.features(["159629.SZ"], fields=None, start_time="2023-01-01", end_time="2023-01-02")
sample_df   
# fields = sample_df.columns.tolist()
# # 构造DataFrame返回
# df_fields = pd.DataFrame({'字段名': fields})
# df_fields


TypeError: 'NoneType' object is not iterable

In [6]:
instruments = D.instruments(market='etf_test')
D.list_instruments(instruments=instruments, start_time='2020-01-01', end_time='2025-12-31', as_list=True)[:6]




['159001.SZ', '159003.SZ', '159005.SZ', '159150.SZ', '159200.SZ', '159201.SZ']

In [16]:
from qlib.data.filter import NameDFilter
nameDFilter = NameDFilter(name_rule_re='5[0-9]{5}.sh')
instruments = D.instruments(market='all', filter_pipe=[nameDFilter])
D.list_instruments(instruments=instruments, start_time='2015-01-01', end_time='2016-02-15', as_list=True)

[]

In [17]:
import qlib
from qlib.data import D
import pandas as pd

# 初始化qlib，假设数据已在~/data/qlib_data/etf_data
qlib.init(provider_uri="~/data/qlib_data/etf_data", region="cn")

# 观察所有ETF的可用代码
instruments = D.list_instruments(market="all", category="ETF")
print("ETF数量:", len(instruments))
print("前5只ETF:", instruments[:5])

# 取某只ETF的部分数据
etf_code = instruments[0]
df = D.features([etf_code], fields=["$open", "$close", "$high", "$low", "$volume"], start_time="2023-03-20", end_time="2023-03-31")
print(df.head())

# 筛选收盘价大于100的记录
filtered = df[df["$close"] > 100]
print("收盘价大于100的记录:")
print(filtered)

# 统计该ETF的成交量均值
print("成交量均值:", df["$volume"].mean())


[1408:MainThread](2025-08-07 00:03:14,514) INFO - qlib.Initialization - [config.py:451] - default_conf: client.
[1408:MainThread](2025-08-07 00:03:14,518) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[1408:MainThread](2025-08-07 00:03:14,519) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': PosixPath('/Users/liyuan/data/qlib_data/etf_data')}


TypeError: BaseProvider.list_instruments() got an unexpected keyword argument 'market'